In [1]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
# If it's for the first time, authentication is required before running this code
# https://developers.google.com/earth-engine/guides/python_install-conda#windows_5
ee.Initialize()

In [3]:
# earth engine location of the subsets of the 10km grid created separately in the script "00_create_10_by_10_grid.R"
# 10km grid shapefile needs to be uploaded to assets in your GEE before running this line
grid = ee.FeatureCollection("projects/ee-akawano/assets/grid_10km")

# google drive associated with your GEE account
output_drive = "landuse_10km"
timezone = "utc"

In [4]:
# MCD12Q1.006 MODIS Land Cover Type Yearly Global 500m
# https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1

maiac = ee.ImageCollection("MODIS/006/MCD12Q1").select("LC_Type3")
maiac_proj = maiac.first().projection()
pixel_res = maiac_proj.nominalScale().getInfo()

In [5]:
# set start_date list, end_date is optional in the filterDate function. 
# If not specified, a 1-millisecond range starting at 'start' is created.
start_date_list = ["2018-01-01", "2019-01-01", "2020-01-01"]
start_date_list

['2018-01-01', '2019-01-01', '2020-01-01']

In [6]:
def create_image(start_date):
    # timezone is default utc in ee.Date.fromYMD
    # Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles.
    # If Urban abd Build-up (value 10), assign 1, otherwise 0
    im = maiac.filterDate(start_date).first().expression("(b('LC_Type3') == 10) ? 1 "
                                     ": 0")
    im = ee.Image(im).set("start_date", start_date.format("yMMdd"))
    return ee.ImageCollection(im)

In [7]:
# For each day, calculate average for each cell grid
def process_image(start_date):
    maiac_images = create_image(start_date)
    maiac_images_reduced = maiac_images.map(lambda daily_im: (
        daily_im.reduceRegions(
            collection = grid,
            reducer = ee.Reducer.mean(),
            crs = maiac_proj,
            scale = pixel_res
        ).map(lambda f: f.set("start_date", daily_im.get("start_date")))))
    grid_maiac = ee.FeatureCollection(maiac_images_reduced).flatten()
    return grid_maiac

In [8]:
# make a list of properties to export 
export_properties = ["grid_id", "mean", "start_date"]
export_properties

['grid_id', 'mean', 'start_date']

In [9]:
# loop using start_date_list and end_date_list
for start_date in start_date_list:
    grid_landuse = process_image(start_date)
    x = "landuse_10km"
    y = start_date
    output_name = "_".join([x, y])
    landuse_task = ee.batch.Export.table.toDrive(
        collection = grid_landuse,
        folder = output_drive,
        description = output_name,
        fileFormat = "CSV",
        selectors = export_properties)
    print(f"saving file as", {output_name})
    landuse_task.start()

saving file as {'landuse_10km_2018-01-01'}
saving file as {'landuse_10km_2019-01-01'}
saving file as {'landuse_10km_2020-01-01'}


In [ ]:
# if you want to cancel tasks:
# https://code.earthengine.google.com/tasks